# Data augmentation

## Ideas
- Change background color
- Image normalization
- zoom range (1.0, 1.2)
- width shift range (10%), height shift range (10%)
- rotation range (±10°)
- brightness range (0.4 1.2)
- blurring images using averaging, median, and gaussian for the dynamic sign.
- morphological operations erosion and dilation, adding noise salt and paper, and sharpening the dataset's images.

## Imports

In [7]:
import cv2
import os
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

In [19]:
from pathlib import Path

## Change background color

In [59]:
# BGR background colors

# usual background colors
light_beige = (201,227,234)
beige = (150,189,210)
dark_beige = (126,174,200)
white_ish = (230,231,233)
white_to_gray = (194,195,197)
blue_gray = (193,189,178)
naturalwhite = (238,240,240)

# unsual
white = (255,255,255)
orange = (33,102,243)
blue = (188,87,35)
yellow = (35,237,252)
red = (39,33,213)

In [37]:
train_or_test = 'train'
color = 'usual_colors'

video_source_path = Path(f"D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/{train_or_test}/")
video_destination_path = Path(f"D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/augmented/{color}/{train_or_test}/")

video_destination_path.mkdir(parents=True, exist_ok=True)

In [63]:
usual_colors = {'beige':beige, 'lightbeige':light_beige, 'darkbeige':dark_beige, 'naturalwhite':naturalwhite, 'whiteish':white_ish, 'whitetogray':white_to_gray, 'bluegray': blue_gray}
unusual_colors = {'white':white, 'orange':orange, 'blue':blue, 'yellow':yellow, 'red':red}

### Test

In [71]:
video_folder_path = "D:/Documentos/Polito/Thesis/Datasets/A3LIS-147_italian/trimmed-10/train/"
# video_file_name = "fal_acqua (online-video-cutter.com).mp4"
video_file_name = "mmr_allegare (online-video-cutter.com).mp4"

size = (720, 576) # (width, height)
fps = 25
color_key = 'beige'

cap = cv2.VideoCapture(os.path.join(video_folder_path, video_file_name))
segmentor = SelfiSegmentation()
img_array = list()
success = True
while success == True:
    success, image = cap.read()
    if success:
        img_out = segmentor.removeBG(image, usual_colors[color_key], cutThreshold=0.8)
        cv2.imshow("image out", img_out)
        cv2.waitKey(200)
        img_array.append(img_out)

out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
cv2.destroyAllWindows()
cap.release()

### Dataset augmentation

In [72]:
size = (720, 576) # (width, height)
fps = 25

color_key = 'beige'
color = usual_colors[color_key]

for video_file_name in video_source_path.glob('*.mp4'):
    label = str(video_file_name.name).split(' ')[0]
    new_video_file_name = f"{label}_{color_key}.mp4"
    cap = cv2.VideoCapture(str(video_file_name))
    segmentor = SelfiSegmentation()
    img_array = list()
    success = True
    while success == True:
        success, image = cap.read()
        if success:
            img_out = segmentor.removeBG(image, color, cutThreshold=0.8)
            img_array.append(img_out)

    out = cv2.VideoWriter(str(video_destination_path/new_video_file_name),cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
    cv2.destroyAllWindows()
    cap.release()

    print(f"Successfully created {new_video_file_name} \n")

Successfully created fal_abbonamento_beige.mp4 

Successfully created fal_abitare_beige.mp4 

Successfully created fal_acqua_beige.mp4 

Successfully created fal_affitto_beige.mp4 

Successfully created fal_allergia_beige.mp4 

Successfully created fal_ambulanza_beige.mp4 

Successfully created fal_amministrazione_beige.mp4 

Successfully created fal_ancona_beige.mp4 

Successfully created fef_abitare_beige.mp4 

Successfully created fef_acqua_beige.mp4 

Successfully created fef_affitto_beige.mp4 

Successfully created fef_allegare_beige.mp4 

Successfully created fef_allergia_beige.mp4 

Successfully created fef_ambulanza_beige.mp4 

Successfully created fef_amministrazione_beige.mp4 

Successfully created fef_andata_beige.mp4 

Successfully created fsf_abitare_beige.mp4 

Successfully created fsf_acqua_beige.mp4 

Successfully created fsf_affitto_beige.mp4 

Successfully created fsf_allegare_beige.mp4 

Successfully created fsf_allergia_beige.mp4 

Successfully created fsf_ambulanza

In [ ]:
img_array = []
for filename in video_path.glob('*.jpg'):
    img = cv2.imread(str(filename))
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

print(size)

out = cv2.VideoWriter('test.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 12, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Tilt (rotation)